## Join the Duet Server the Data Owner 1 connected to

In [34]:
import pandas as pd

In [1]:
import syft as sy
duet1 = sy.duet("3f57845e06356684367141bc05b590a4")

🎤  🎸  ♪♪♪ Joining Duet ♫♫♫  🎻  🎹

♫♫♫ > DISCLAIMER: Duet is an experimental feature currently in beta.
♫♫♫ > Use at your own risk.


    > ❤️ Love Duet? Please consider supporting our community!
    > https://github.com/sponsors/OpenMined

♫♫♫ > Punching through firewall to OpenGrid Network Node at:
♫♫♫ > http://ec2-18-218-7-180.us-east-2.compute.amazonaws.com:5000
♫♫♫ >
♫♫♫ > ...waiting for response from OpenGrid Network... 
♫♫♫ > DONE!

♫♫♫ > STEP 1: Send the following Duet Client ID to your duet partner!
♫♫♫ > Duet Client ID: 8da0e35e487bb8807f42215a2a27bc70

♫♫♫ > ...waiting for partner to connect...


C:\Users\dylan\anaconda3\envs\new_pysyft\lib\site-packages\aiortc\rtcdtlstransport.py:211: CryptographyDeprecationWarning: This version of cryptography contains a temporary pyOpenSSL fallback path. Upgrade pyOpenSSL now.
  _openssl_assert(lib.SSL_CTX_use_certificate(ctx, self._cert._x509) == 1)  # type: ignore
C:\Users\dylan\anaconda3\envs\new_pysyft\lib\site-packages\aiortc\rtcdtlstransport.py:186: CryptographyDeprecationWarning: This version of cryptography contains a temporary pyOpenSSL fallback path. Upgrade pyOpenSSL now.
  value=certificate_digest(self._cert._x509),  # type: ignore



♫♫♫ > CONNECTED!


## Join the Duet Server the Data Owner 2 connected to

In [13]:
duet2 = sy.duet("2384833fcfdc648673be866295ffaf16")

🎤  🎸  ♪♪♪ Joining Duet ♫♫♫  🎻  🎹

♫♫♫ > DISCLAIMER: Duet is an experimental feature currently in beta.
♫♫♫ > Use at your own risk.


    > ❤️ Love Duet? Please consider supporting our community!
    > https://github.com/sponsors/OpenMined

♫♫♫ > Punching through firewall to OpenGrid Network Node at:
♫♫♫ > http://ec2-18-218-7-180.us-east-2.compute.amazonaws.com:5000
♫♫♫ >
♫♫♫ > ...waiting for response from OpenGrid Network... 
♫♫♫ > DONE!

♫♫♫ > STEP 1: Send the following Duet Client ID to your duet partner!
♫♫♫ > Duet Client ID: 0d921f9becf04aaca60b9b9ba4780e17

♫♫♫ > ...waiting for partner to connect...


C:\Users\dylan\anaconda3\envs\new_pysyft\lib\site-packages\aiortc\rtcdtlstransport.py:211: CryptographyDeprecationWarning: This version of cryptography contains a temporary pyOpenSSL fallback path. Upgrade pyOpenSSL now.
  _openssl_assert(lib.SSL_CTX_use_certificate(ctx, self._cert._x509) == 1)  # type: ignore
C:\Users\dylan\anaconda3\envs\new_pysyft\lib\site-packages\aiortc\rtcdtlstransport.py:186: CryptographyDeprecationWarning: This version of cryptography contains a temporary pyOpenSSL fallback path. Upgrade pyOpenSSL now.
  value=certificate_digest(self._cert._x509),  # type: ignore



♫♫♫ > CONNECTED!


In [2]:
duet1.store.pandas

,ID,Tags,Description,object_type
0,<UID: 46037f1f37794325905f550c2951cc7a>,[DO1 training X data],number of samples and features,<class 'torch.Tensor'>
1,<UID: b5b0998d9dc14891a63d2f163a26c555>,[DO1 training y data],number of samples and features,<class 'torch.Tensor'>


In [14]:
duet2.store.pandas

,ID,Tags,Description,object_type
0,<UID: 517253510c5c41d78a39921bbd033707>,[DO2 training X data],number of samples and features,<class 'torch.Tensor'>
1,<UID: e52bb7880b504d3798a5d493771c32b0>,[DO1 training y data],number of samples and features,<class 'torch.Tensor'>


## Linear regression

In [18]:
data1_ptr = duet1.store[0]
target1_ptr = duet1.store[1]
data2_ptr = duet2.store[0]
target2_ptr = duet2.store[1]

print(data1_ptr)
print(target1_ptr)
print(data2_ptr)
print(target2_ptr)

### Create Base Model

In [42]:
import torch
import torch.nn.functional as F
import torch.nn as nn 
from torch import optim
from torch.utils.data import Dataset, DataLoader

In [5]:
in_dim = 39
out_dim = 1

In [63]:
class SyNet(sy.Module):
    def __init__(self, torch_ref):
        super(SyNet, self).__init__(torch_ref=torch_ref)
        self.linear = self.torch_ref.nn.Linear(in_dim, out_dim)

    def forward(self, x):
        x = self.linear(x)
        return x

In [77]:
class SyNet(sy.Module):
    def __init__(self, torch_ref):
        super(SyNet, self).__init__(torch_ref=torch_ref)
        
        self.layer_1 = self.torch_ref.nn.Linear(39, 31)
        self.layer_out = self.torch_ref.nn.Linear(31, 1)
        # Define proportion or neurons to dropout
        self.dropout_1 = self.torch_ref.nn.Dropout(0.12409392594394411)
        self.relu = self.torch_ref.nn.ReLU()
    
    def forward(self, inputs):
        x = self.dropout_1(self.relu(self.layer_1(inputs)))
        x = self.layer_out(x)

        return (x)

    def predict(self, test_inputs):
        x = self.relu(self.layer_1(test_inputs))
        x = self.layer_out(x)

        return (x)

### Training

In [147]:
def train(iterations, model, torch_ref, optim, data_ptr, target_ptr):

    #criterion = torch_ref.nn.functional.poisson_nll_loss(log_input= True, full= True)

    losses = []

    for i in range(iterations):

        optim.zero_grad()

        output = model(data_ptr)

        #loss = torch_ref.nn.functional.nll_loss(output, target_ptr)
        loss = torch_ref.nn.functional.poisson_nll_loss(output, target_ptr, log_input= True, full= True)   
        #loss = torch_ref.nn.functional.mse_loss(output, target_ptr)
        #loss = criterion(output, target_ptr)

        loss_item = loss.item()

        loss_value = loss_item.get(
            reason="To evaluate training progress",
            request_block=True,
            timeout_secs=5,
        )

        print("Epoch", i, "loss", loss_value)

        losses.append(loss_value)

        loss.backward()

        optim.step()

    return losses

#### Send one copy of the model to each data owner or client and train them remotely one by one

In [153]:
import torch as th
import numpy as np

In [154]:
base_model = SyNet(torch)

Train on Data Owner 1 data

In [155]:
remote_model1 = base_model.send(duet1)

In [156]:
remote_torch1 = duet1.torch
params = remote_model1.parameters()
optim1 = remote_torch1.optim.Adam(params=params, lr=0.1)

In [157]:
iteration = 25
losses = train(iteration, remote_model1, remote_torch1, optim1, data1_ptr, target1_ptr)

Epoch 0 loss 1.3621643781661987
Epoch 1 loss 0.30999502539634705
Epoch 2 loss 0.24786120653152466
Epoch 3 loss 0.3490721583366394
Epoch 4 loss 0.4000135362148285
Epoch 5 loss 0.39987507462501526
Epoch 6 loss 0.3644452393054962
Epoch 7 loss 0.3117058277130127
Epoch 8 loss 0.25791317224502563
Epoch 9 loss 0.22383953630924225
Epoch 10 loss 0.23352254927158356
Epoch 11 loss 0.2754928171634674
Epoch 12 loss 0.27155810594558716
Epoch 13 loss 0.24048081040382385
Epoch 14 loss 0.22326281666755676
Epoch 15 loss 0.22320111095905304
Epoch 16 loss 0.2308705449104309
Epoch 17 loss 0.23766162991523743
Epoch 18 loss 0.2405579388141632
Epoch 19 loss 0.23858143389225006
Epoch 20 loss 0.23393642902374268
Epoch 21 loss 0.2276202142238617
Epoch 22 loss 0.2213977426290512
Epoch 23 loss 0.2180577963590622
Epoch 24 loss 0.21901576220989227


Train on Data Owner 2 data

In [15]:
remote_model2 = base_model.send(duet2)

In [16]:
remote_torch2 = duet2.torch
params = remote_model2.parameters()
optim2 = remote_torch2.optim.Adam(params=params, lr=0.1)

In [19]:
iteration = 10
losses = train(iteration, remote_model2, remote_torch2, optim2, data2_ptr, target2_ptr)

Epoch 0 loss 0.06821049004793167
Epoch 1 loss 0.2506844997406006
Epoch 2 loss 0.06862416118383408
Epoch 3 loss 0.2005573809146881
Epoch 4 loss 0.1580241322517395
Epoch 5 loss 0.06731531769037247
Epoch 6 loss 0.07882550358772278
Epoch 7 loss 0.1320706307888031
Epoch 8 loss 0.12076710164546967
Epoch 9 loss 0.07351528853178024


### Averaging Model Updates

Ideally, there will be a coordinator server with a secure aggreagtor who will get the model updates from different clients and make an aggregation. For the case of simplicity, in this example we will make the Data Sceintist server work as the coordinator.

### Little sanity check!

In [20]:
param1 = remote_model1.parameters().get(request_block=True)
param2 = remote_model2.parameters().get(request_block=True)

print("Base Model parameters:")
print(base_model.parameters())
print()

print("Remote model1 parameters:")
print(param1)
print()

print("Remote model2 parameters:")
print(param2)

Base Model parameters:
[Parameter containing:
tensor([[ 0.1282, -0.0424, -0.0517, -0.0962,  0.0536,  0.0536,  0.0197, -0.0636,
          0.1002, -0.0063, -0.1552, -0.1015,  0.0564,  0.0056, -0.0507, -0.1325,
          0.0990,  0.0093, -0.1175,  0.0065,  0.0067,  0.0283, -0.0927, -0.1017,
          0.0835, -0.0662,  0.0981, -0.1420, -0.0226, -0.0180,  0.0489,  0.0634,
          0.0215, -0.1569, -0.0234, -0.0762,  0.1394,  0.1230,  0.0224]],
       requires_grad=True), Parameter containing:
tensor([0.0623], requires_grad=True)]

Remote model1 parameters:
[Parameter containing:
tensor([[ 0.0863,  0.1108, -0.0463, -0.0837,  0.0287,  0.1884, -0.0090, -0.1014,
          0.0041,  0.0450, -0.0152,  0.0004, -0.0066,  0.0168,  0.0361, -0.0205,
         -0.0344, -0.0079,  0.0666,  0.0008, -0.0030,  0.1087,  0.0626,  0.0493,
          0.0178,  0.0166,  0.0191,  0.0586,  0.0242,  0.1258,  0.0876,  0.0497,
         -0.0122,  0.1242,  0.0277,  0.0190,  0.0112,  0.0140, -0.0087]],
       requires_grad

As you can see, the remote model paramter values are different from the base model paramter values. That means the remote copies of our base model got trained and updated.

In [21]:
remote_model1_updates = remote_model1.get(
    request_block=True
).state_dict()

print(remote_model1_updates)

OrderedDict([('linear.weight', tensor([[ 0.0863,  0.1108, -0.0463, -0.0837,  0.0287,  0.1884, -0.0090, -0.1014,
          0.0041,  0.0450, -0.0152,  0.0004, -0.0066,  0.0168,  0.0361, -0.0205,
         -0.0344, -0.0079,  0.0666,  0.0008, -0.0030,  0.1087,  0.0626,  0.0493,
          0.0178,  0.0166,  0.0191,  0.0586,  0.0242,  0.1258,  0.0876,  0.0497,
         -0.0122,  0.1242,  0.0277,  0.0190,  0.0112,  0.0140, -0.0087]])), ('linear.bias', tensor([0.0314]))])


In [22]:
remote_model2_updates = remote_model2.get(
    request_block=True
).state_dict()

print(remote_model2_updates)

OrderedDict([('linear.weight', tensor([[ 0.0854,  0.1097, -0.0482, -0.0832,  0.0293,  0.1900, -0.0088, -0.1019,
         -0.0002,  0.0510, -0.0169,  0.0018, -0.0097,  0.0157,  0.0288, -0.0238,
         -0.0314, -0.0100,  0.0600, -0.0005, -0.0035,  0.1077,  0.0561,  0.0603,
          0.0270,  0.0261,  0.0306,  0.0858,  0.0256,  0.1213,  0.0763,  0.0500,
         -0.0213,  0.1234,  0.0299,  0.0287,  0.0102,  0.0176, -0.0044]])), ('linear.bias', tensor([0.0314]))])


In [23]:
from collections import OrderedDict

Let's do the aggregation of the weights. In this example, we will just calculate the average of corresponding weights from each model.

In [24]:
avg_updates = OrderedDict()
avg_updates["linear.weight"] = (
    remote_model1_updates["linear.weight"] + remote_model2_updates["linear.weight"]
) / 2
avg_updates["linear.bias"] = (
    remote_model1_updates["linear.bias"] + remote_model2_updates["linear.bias"]
) / 2

print(avg_updates)

OrderedDict([('linear.weight', tensor([[ 8.5854e-02,  1.1021e-01, -4.7242e-02, -8.3441e-02,  2.8979e-02,
          1.8916e-01, -8.9127e-03, -1.0162e-01,  1.9264e-03,  4.7996e-02,
         -1.6030e-02,  1.0802e-03, -8.1175e-03,  1.6279e-02,  3.2430e-02,
         -2.2112e-02, -3.2899e-02, -8.9505e-03,  6.3325e-02,  1.3219e-04,
         -3.2738e-03,  1.0820e-01,  5.9352e-02,  5.4828e-02,  2.2410e-02,
          2.1305e-02,  2.4866e-02,  7.2193e-02,  2.4896e-02,  1.2355e-01,
          8.1971e-02,  4.9882e-02, -1.6787e-02,  1.2376e-01,  2.8834e-02,
          2.3831e-02,  1.0697e-02,  1.5796e-02, -6.5223e-03]])), ('linear.bias', tensor([0.0314]))])


### Load aggregated weights

In [25]:
combined_model = SyNet(torch)

In [26]:
combined_model.load_state_dict(avg_updates)

In [27]:
del avg_updates, remote_model1_updates, remote_model2_updates

In [30]:
df_X_val_0 = pd.read_csv('ETL/X_val_0.csv')
df_y_val_0 = pd.read_csv('ETL/y_val_0.csv')
X_val_0_np = df_X_val_0.to_numpy()
y_val_0_np = df_y_val_0.to_numpy()
X_val_0 = th.from_numpy(X_val_0_np).float()
y_val_0 = th.from_numpy(y_val_0_np).float()

In [32]:
preds = []
with torch.no_grad():
    for i in range(len(X_val_0)):
        sample = X_val_0[i]
        y_hat = combined_model(sample)

        print(f"Prediction: {y_hat.item()} Ground Truth: {y_val_0[i].item()}")
        preds.append(y_hat)

Prediction: 0.25279733538627625 Ground Truth: 0.0
Prediction: 0.018277831375598907 Ground Truth: 0.0
Prediction: 0.016164973378181458 Ground Truth: 0.0
Prediction: 0.06737774610519409 Ground Truth: 0.0
Prediction: 0.18245826661586761 Ground Truth: 0.0
Prediction: 0.10276442021131516 Ground Truth: 0.0
Prediction: 0.168299600481987 Ground Truth: 0.0
Prediction: 0.03854905068874359 Ground Truth: 1.0
Prediction: 0.0680687204003334 Ground Truth: 0.0
Prediction: 0.08793748915195465 Ground Truth: 0.0
Prediction: 0.06875209510326385 Ground Truth: 0.0
Prediction: 0.19969308376312256 Ground Truth: 0.0
Prediction: 0.14694836735725403 Ground Truth: 0.0
Prediction: 0.17181631922721863 Ground Truth: 2.0
Prediction: 0.11202537268400192 Ground Truth: 0.0
Prediction: 0.11018943786621094 Ground Truth: 0.0
Prediction: 0.14999771118164062 Ground Truth: 0.0
Prediction: 0.1069117933511734 Ground Truth: 0.0
Prediction: 0.10453125834465027 Ground Truth: 0.0
Prediction: 0.12687912583351135 Ground Truth: 0.0
Pr

In [33]:
preds = []
with torch.no_grad():
    for i in range(len(X_val_0)):
        sample = X_val_0[i]
        y_hat = base_model(sample)

        print(f"Prediction: {y_hat.item()} Ground Truth: {y_val_0[i].item()}")
        preds.append(y_hat)

Prediction: -0.002812005579471588 Ground Truth: 0.0
Prediction: 0.23667319118976593 Ground Truth: 0.0
Prediction: -0.137446328997612 Ground Truth: 0.0
Prediction: 0.025279030203819275 Ground Truth: 0.0
Prediction: 0.1057082861661911 Ground Truth: 0.0
Prediction: 0.02055583894252777 Ground Truth: 0.0
Prediction: 0.2783677577972412 Ground Truth: 0.0
Prediction: 0.1261349767446518 Ground Truth: 1.0
Prediction: -0.018150418996810913 Ground Truth: 0.0
Prediction: 0.12427321821451187 Ground Truth: 0.0
Prediction: -0.15800915658473969 Ground Truth: 0.0
Prediction: 0.1691092848777771 Ground Truth: 0.0
Prediction: 0.04366951435804367 Ground Truth: 0.0
Prediction: 0.24116000533103943 Ground Truth: 2.0
Prediction: 0.15667478740215302 Ground Truth: 0.0
Prediction: 0.1729632169008255 Ground Truth: 0.0
Prediction: 0.027431204915046692 Ground Truth: 0.0
Prediction: 0.021964959800243378 Ground Truth: 0.0
Prediction: 0.21308061480522156 Ground Truth: 0.0
Prediction: 0.17932026088237762 Ground Truth: 0.

## Comparison to classical linear regression on centralised data

In [ ]:
import torch
import numpy as np

in_dim = 1
out_dim = 1


class ClassicalLR(torch.nn.Module):
    def __init__(self, torch):
        super(ClassicalLR, self).__init__()
        self.linear = torch.nn.Linear(in_dim, out_dim)

    def forward(self, x):
        x = self.linear(x)
        return x


classical_model = ClassicalLR(torch)

In [ ]:
data = torch.FloatTensor(
    np.array([5, 15, 25, 35, 45, 55, 60, 65, 75, 85, 95]).reshape(-1, 1)
)
target = torch.FloatTensor(
    np.array([5, 10, 15, 22, 30, 38, 35, 40, 45, 55, 60]).reshape(-1, 1)
)

In [ ]:
def classic_train(iterations, model, torch, optim, data, target, criterion):

    losses = []

    for i in range(iterations):

        optim.zero_grad()

        output = model(data)

        loss = criterion(output, target)

        loss_item = loss.item()

        if i % 10 == 0:
            print("Epoch", i, "loss", loss_item)

        losses.append(loss_item)

        loss.backward()

        optim.step()

    return losses

In [ ]:
params = classical_model.parameters()
optim = torch.optim.Adam(params=params, lr=0.1)
criterion = torch.nn.MSELoss()

In [ ]:
iteration = 100
losses = classic_train(
    iteration, classical_model, torch, optim, data, target, criterion
)

In [ ]:
test_data = th.FloatTensor(np.array([17, 25, 32, 50, 80]).reshape(-1, 1))
test_target = th.FloatTensor(np.array([12, 15, 20, 30, 50]).reshape(-1, 1))

In [ ]:
preds = []
with torch.no_grad():
    for i in range(len(test_data)):
        sample = test_data[i]
        y_hat = classical_model(sample)

        print(f"Prediction: {y_hat.item()} Ground Truth: {test_target[i].item()}")
        preds.append(y_hat)